In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [6]:
list_paddings = [
    #'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                 #'rnd_padding', 
                 'aug_padding']
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
architecture = "only_denses"

### Loading stuff

In [3]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [4]:
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_2 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 76)                0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = None, kernel_size=None, pool_size=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_8 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_9 (Dropout)          (None, 76)                0         
__________

828/828 [==============================] - 12s 15ms/step - loss: 0.4907 - acc: 0.2961 - val_loss: 0.2052 - val_acc: 0.1114

Epoch 00030: val_acc did not improve from 0.11142
Epoch 31/200
828/828 [==============================] - 12s 15ms/step - loss: 0.4884 - acc: 0.2979 - val_loss: 0.2039 - val_acc: 0.1146

Epoch 00031: val_acc improved from 0.11142 to 0.11456, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/0/weights-improvement-031-0.1146.hdf5
Epoch 32/200
828/828 [==============================] - 12s 15ms/step - loss: 0.4863 - acc: 0.2993 - val_loss: 0.2029 - val_acc: 0.1146

Epoch 00032: val_acc did not improve from 0.11456
Epoch 33/200
828/828 [==============================] - 12s 15ms/step - loss: 0.4720 - acc: 0.3023 - val_loss: 0.1981 - val_acc: 0.1154

Epoch 00033: val_acc improved from 0.11456 to 0.11540, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug

828/828 [==============================] - 12s 15ms/step - loss: 0.3406 - acc: 0.3549 - val_loss: 0.1329 - val_acc: 0.1518

Epoch 00059: val_acc did not improve from 0.15286
Epoch 60/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3415 - acc: 0.3542 - val_loss: 0.1314 - val_acc: 0.1584

Epoch 00060: val_acc improved from 0.15286 to 0.15840, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/0/weights-improvement-060-0.1584.hdf5
Epoch 61/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3319 - acc: 0.3586 - val_loss: 0.1350 - val_acc: 0.1507

Epoch 00061: val_acc did not improve from 0.15840
Epoch 62/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3305 - acc: 0.3563 - val_loss: 0.1234 - val_acc: 0.1669

Epoch 00062: val_acc improved from 0.15840 to 0.16688, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug


Epoch 00093: val_acc did not improve from 0.18079
Epoch 94/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2631 - acc: 0.3853 - val_loss: 0.0913 - val_acc: 0.1824

Epoch 00094: val_acc improved from 0.18079 to 0.18236, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/0/weights-improvement-094-0.1824.hdf5
Epoch 95/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2544 - acc: 0.3875 - val_loss: 0.0975 - val_acc: 0.1737

Epoch 00095: val_acc did not improve from 0.18236
Epoch 96/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2547 - acc: 0.3864 - val_loss: 0.0949 - val_acc: 0.1810

Epoch 00096: val_acc did not improve from 0.18236
Epoch 97/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2745 - acc: 0.3763 - val_loss: 0.0929 - val_acc: 0.1853

Epoch 00097: val_acc improved from 0.18236 to 0.18529, saving model to /home/angela/padding_EBI

828/828 [==============================] - 12s 15ms/step - loss: 0.2145 - acc: 0.4045 - val_loss: 0.0943 - val_acc: 0.1725

Epoch 00135: val_acc did not improve from 0.18717
Epoch 136/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2134 - acc: 0.4015 - val_loss: 0.0873 - val_acc: 0.1802

Epoch 00136: val_acc did not improve from 0.18717
Epoch 137/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2145 - acc: 0.4041 - val_loss: 0.0993 - val_acc: 0.1801

Epoch 00137: val_acc did not improve from 0.18717
Epoch 138/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2134 - acc: 0.4035 - val_loss: 0.0909 - val_acc: 0.1803

Epoch 00138: val_acc did not improve from 0.18717
Epoch 139/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2125 - acc: 0.4027 - val_loss: 0.1048 - val_acc: 0.1769

Epoch 00139: val_acc did not improve from 0.18717
Epoch 140/200
828/828 [==============================] - 12s 15ms/step - l

...

828/828 [==============================] - 12s 15ms/step - loss: 0.5016 - acc: 0.2911 - val_loss: 0.2166 - val_acc: 0.1059

Epoch 00023: val_acc improved from 0.10421 to 0.10588, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/5/weights-improvement-023-0.1059.hdf5
Epoch 24/200
828/828 [==============================] - 12s 15ms/step - loss: 0.4957 - acc: 0.2937 - val_loss: 0.2152 - val_acc: 0.1065

Epoch 00024: val_acc improved from 0.10588 to 0.10651, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/5/weights-improvement-024-0.1065.hdf5
Epoch 25/200
828/828 [==============================] - 12s 15ms/step - loss: 0.4880 - acc: 0.2973 - val_loss: 0.2130 - val_acc: 0.1086

Epoch 00025: val_acc improved from 0.10651 to 0.10860, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/5/weights-improvement-

828/828 [==============================] - 12s 15ms/step - loss: 0.3573 - acc: 0.3487 - val_loss: 0.1502 - val_acc: 0.1486

Epoch 00056: val_acc improved from 0.14490 to 0.14857, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/5/weights-improvement-056-0.1486.hdf5
Epoch 57/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3556 - acc: 0.3467 - val_loss: 0.1509 - val_acc: 0.1455

Epoch 00057: val_acc did not improve from 0.14857
Epoch 58/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3500 - acc: 0.3493 - val_loss: 0.1492 - val_acc: 0.1455

Epoch 00058: val_acc did not improve from 0.14857
Epoch 59/200
828/828 [==============================] - 12s 15ms/step - loss: 0.3567 - acc: 0.3492 - val_loss: 0.1421 - val_acc: 0.1575

Epoch 00059: val_acc improved from 0.14857 to 0.15746, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug

828/828 [==============================] - 12s 15ms/step - loss: 0.2796 - acc: 0.3771 - val_loss: 0.1162 - val_acc: 0.1629

Epoch 00094: val_acc did not improve from 0.16520
Epoch 95/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2806 - acc: 0.3776 - val_loss: 0.1190 - val_acc: 0.1611

Epoch 00095: val_acc did not improve from 0.16520
Epoch 96/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2773 - acc: 0.3767 - val_loss: 0.1757 - val_acc: 0.1520

Epoch 00096: val_acc did not improve from 0.16520
Epoch 97/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2769 - acc: 0.3790 - val_loss: 0.1364 - val_acc: 0.1608

Epoch 00097: val_acc did not improve from 0.16520
Epoch 98/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2801 - acc: 0.3788 - val_loss: 0.1528 - val_acc: 0.1588

Epoch 00098: val_acc did not improve from 0.16520
Epoch 99/200
828/828 [==============================] - 12s 15ms/step - loss: 


Epoch 00133: val_acc did not improve from 0.17284
Epoch 134/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2406 - acc: 0.3959 - val_loss: 0.0986 - val_acc: 0.1692

Epoch 00134: val_acc did not improve from 0.17284
Epoch 135/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2318 - acc: 0.4007 - val_loss: 0.1026 - val_acc: 0.1685

Epoch 00135: val_acc did not improve from 0.17284
Epoch 136/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2367 - acc: 0.3982 - val_loss: 0.0944 - val_acc: 0.1694

Epoch 00136: val_acc did not improve from 0.17284
Epoch 137/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2345 - acc: 0.3983 - val_loss: 0.0849 - val_acc: 0.1790

Epoch 00137: val_acc improved from 0.17284 to 0.17901, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/aug_padding/5/weights-improvement-137-0.1790.hdf5
Epoch 138/200
828/828 [==================

828/828 [==============================] - 12s 15ms/step - loss: 0.2095 - acc: 0.4103 - val_loss: 0.0894 - val_acc: 0.1702

Epoch 00174: val_acc did not improve from 0.18864
Epoch 175/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2116 - acc: 0.4094 - val_loss: 0.0695 - val_acc: 0.1800

Epoch 00175: val_acc did not improve from 0.18864
Epoch 176/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2064 - acc: 0.4105 - val_loss: 0.0922 - val_acc: 0.1739

Epoch 00176: val_acc did not improve from 0.18864
Epoch 177/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2050 - acc: 0.4093 - val_loss: 0.0874 - val_acc: 0.1740

Epoch 00177: val_acc did not improve from 0.18864
Epoch 178/200
828/828 [==============================] - 12s 15ms/step - loss: 0.2083 - acc: 0.4099 - val_loss: 0.0880 - val_acc: 0.1740

Epoch 00178: val_acc did not improve from 0.18864
Epoch 179/200
828/828 [==============================] - 12s 15ms/step - l

In [ ]:
# BORRAR CUANDO COMPRUEBE QUE LO OTRO VA BIEN

task = "task2/"
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else:
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        folder_cp = ''.join(string for string in [folder, task, model_type, '/', str(idx)]) 
        #print(folder_cp)
        if not os.path.exists(os.path.join(absPath, 'data/checkpoint/', folder_cp)):
            os.makedirs(''.join(string for string in [absPath, 'data/checkpoint/', folder_cp]))

        callbacks_list = calling_callbacks(folder_cp, folder, model_type, None, None, None, 
                                   None, False, False, False, False)
        folder_task1 =  ''.join(string for string in [folder, task])
        model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
        #writing log file 
        log_file = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, 'log_file.txt' ]) 
        f = open(log_file, 'a+')
        print('Model type: %s \n' % model_type, file=f)
        print('Fold: %i \n' % idx, file=f)
 
        start = time.time()
        formatted_time = datetime.datetime.now()
        print('Starting time: %s \n' % formatted_time, file=f)
        history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len_train/batch_size),
                              validation_steps=int(len_val/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
        end = time.time()
        formatted_endtime = datetime.datetime.now()
        print('Finishing time: %s \n' % formatted_endtime, file=f)
        count_time(start, end, folder, model_type)
        saving_results(history, model_type, folder, idx, True)
        f.close()